# Demonstrating semantic nature of OpenAI embeddings

In [204]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [205]:
import os
import openai

# Set OpenAI API credentials
openai.api_type = "azure"
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("BASE_URL")
openai.api_version = "2023-07-01-preview"
deployment_name = os.getenv("DEPLOYMENT_NAME")

In [206]:
# Here is list o words that we want to create embeddings for and store in faiss
texts = [
    "Král: Mužský panovník, který typicky získává svou pozici dědičně a vládne nad královstvím nebo impériem. Králové sehráli v historii významnou roli a byli považováni za mocné postavy s možností činit důležitá rozhodnutí, která ovlivňují životy jejich poddaných.",
    "Královna: Ženský panovník, který typicky získává svou pozici dědičně nebo svatbou s králem. Královny sehráli v historii významnou roli a byli považováni za mocné postavy s možností ovlivňovat rozhodnutí králů. Královny se zabývaly politikou, uměním i filantropií.",
    "Muž: Samec člověka, mužská lidská bytost, která se odlišuje fyzickými charakteristikami jako je hlubší hlas, přítomnost vousů, větší výška a svalová hmota než u žen.",
    "Žena: Samice člověka, ženská lidská bytost, která se odlišuje fyzickými charakteristikami jako jsou prsa, širší boky a vyšší hlas než u mužů.",
    "Pes: domestikovaná šelma, která provází člověka minimálně 14 tisíc let. Pes je považován za nejlepšího přítele člověka.",
    "Lev: savec čeledi kočkovitých a jeden z pěti druhů velkých koček rodu Panthera. Lev je po tygrovi druhá největší kočkovitá šelma. ",
    "Guláš: pokrm z kousků masa dušeného na cibulce s paprikou. Guláš je původně maďarský, ale je oblíbený i v Česku a na Slovensku. Existuje mnoho druhů guláše, například bramborový, segedínský nebo havlův.",
    "Hrad: opevněné feudální sídlo1, které se stavělo většinou v rozmezí 11. až 16. století",
    "Kuželky: hovorové označení pro sportovní hru, při které hráč hází kouli na devět dřevěných kuželek postavených do tvaru čtverce1",
]

# Create a pandas DataFrame from the list of words
import pandas as pd

df = pd.DataFrame(texts, columns=["text"])

def get_embedding(text):
   return openai.Embedding.create(input = [text], deployment_id=deployment_name)['data'][0]['embedding']

df['embedding'] = df['text'].apply(lambda x: get_embedding(x))

In [226]:
from openai.embeddings_utils import cosine_similarity
import numpy as np

def get_similar(source_texts, source_embeddings, target_embedding):
    df = pd.DataFrame(source_texts, columns=['text'])
    df['embedding'] = source_embeddings
    df['similarities'] = df['embedding'].apply(lambda x: cosine_similarity(x, target_embedding))
    return df.sort_values(by='similarities', ascending=False).drop(columns=['embedding'])

get_similar(df['text'], df['embedding'], get_embedding('Karel IV.'))

,text,similarities
0,"Král: Mužský panovník, který typicky získává s...",0.826982
1,"Královna: Ženský panovník, který typicky získá...",0.820007
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.791113
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.787196
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.786764
4,"Pes: domestikovaná šelma, která provází člověk...",0.763578
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.759550
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.752823
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.751189


In [209]:
get_similar(df['text'], df['embedding'], get_embedding('woman sitting on throne'))

,text,similarities
1,"Královna: Ženský panovník, který typicky získá...",0.820404
0,"Král: Mužský panovník, který typicky získává s...",0.806356
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.780668
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.762109
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.757530
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.744676
4,"Pes: domestikovaná šelma, která provází člověk...",0.737916
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.726672
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.709054


In [210]:
get_similar(df['text'], df['embedding'], get_embedding('Bowling is a game in which players roll balls at objects or marks. Bowling can be played indoors on a wooden alley with pins or outdoors on a green with bowls.'))

,text,similarities
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.772278
4,"Pes: domestikovaná šelma, která provází člověk...",0.707202
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.705637
0,"Král: Mužský panovník, který typicky získává s...",0.704169
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.703992
1,"Královna: Ženský panovník, který typicky získá...",0.703222
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.701407
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.693608
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.686470


In [225]:
get_similar(df['text'], df['embedding'], get_embedding('食物'))

,text,similarities
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.751972
4,"Pes: domestikovaná šelma, která provází člověk...",0.740163
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.728779
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.713422
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.712763
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.711843
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.704704
1,"Královna: Ženský panovník, který typicky získá...",0.688390
0,"Král: Mužský panovník, který typicky získává s...",0.685373


In [221]:
# Vector calculations
positive_text = ["Král je dědičný titul panovníka monarchie. V minulosti byl král nejvyšší autoritou v zemi a měl mnoho privilegi",
                 "Žena je podstatné jméno ženského rodu, které označuje dospělou samici člověka. V obecném jazyce se slovo žena používá také jako synonymum pro paní nebo dáma"]
negative_text = ["Muž je podstatné jméno mužského rodu, které označuje dospělého samce člověka. V obecném jazyce se slovo muž používá také jako synonymum pro chlap nebo kluk."]
vector = np.zeros(1536)
vector = sum([np.array(get_embedding(text)) for text in positive_text])
vector = vector + sum([-np.array(get_embedding(text)) for text in negative_text])

get_similar(df['text'], df['embedding'], vector)

,text,similarities
1,"Královna: Ženský panovník, který typicky získá...",0.839392
0,"Král: Mužský panovník, který typicky získává s...",0.806693
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.756048
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.741525
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.729424
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.720193
4,"Pes: domestikovaná šelma, která provází člověk...",0.711233
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.708566
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.688603
